In [26]:
#import libraries
import pandas as pd
import numpy as np
import nltk
#from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import demoji
import pickle

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AishwaryaBhangale\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
%%time
tweets_complete = pd.read_table('conv.txt',sep=',')

Wall time: 2.96 ms


In [76]:
tweets_complete.shape

(100, 2)

In [4]:
tweets_complete[0:2]

,text_x,text_y
0,@sprintcare is the worst customer service,@115712 Hello! We never like our customers to ...
1,@sprintcare is the worst customer service,@115712 I would love the chance to review the ...


## Data Preprocessing

In [66]:
tweets = tweets_complete.head(20)
tweets

,text_x,text_y
0,@sprintcare is the worst customer service,@115712 Hello! We never like our customers to ...
1,@sprintcare is the worst customer service,@115712 I would love the chance to review the ...
2,@sprintcare is the worst customer service,@115712 Can you please send us a private messa...
3,@115714 y’all lie about your “great” connectio...,@115713 H there! We'd definitely like to work ...
4,"@115714 whenever I contact customer support, t...",@115715 Please send me a private message so th...
5,actually that's a broken link you sent me and ...,@115716 The information pertaining to the acco...
6,"Yo @Ask_Spectrum, your customer service reps a...","@115717 Hello, My apologies for any frustratio..."
7,My picture on @Ask_Spectrum pretty much every ...,@115718 I apologize for the inconvenience. I w...
8,somebody from @VerizonSupport please help meee...,@115719 Help has arrived! We are sorry to see ...
9,@VerizonSupport My friend is without internet ...,@115720 Have your friend message us.\r\n^ACM


In [77]:
with open(r'C:\Users\AishwaryaBhangale\Documents\GitHub\nlp-coe\twitter-conversational-chatbot\Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    for emot in Emoji_Dict:
        text = re.sub(r'(' +emot+ ')', "".join(Emoji_Dict[emot].replace(":"," ")),text)
    return text

def remove_URL(text):
    return re.sub(r"http\S+", "", str(text))

In [78]:
#Define stop words
stop_words = set(stopwords.words('english'))
#define special characters list
special_characters = ['!','#','$','%', '&','@','[',']',' ',']','_']

tweets['text_x_clean'] = ""
tweets['text_y_clean'] = ""

for i in range(len(tweets)):

    #################################################################################################################
    
    
    
    tweets['text_x_clean'][i] = tweets['text_x'][i].lower()
    
    #Conversion of Emojis to Text
    tweets['text_x_clean'][i] = convert_emojis_to_word(tweets['text_x_clean'][i])
    
    #Removal of URLS
    pattern=r'(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))';
    match = re.findall(pattern, tweets['text_x_clean'][i])
    for m in match:
        url = m[0]
        tweets['text_x_clean'][i] = tweets['text_x_clean'][i].replace(url, '')
    
    #Removal of HTTPS
    tweets['text_x_clean'][i] = remove_URL(tweets['text_x_clean'][i])
    
    
    #We don't need the following steps of text data preprocessing for deep learning models 
    '''
    #Remove Punctuation --remove
    tokenizer = RegexpTokenizer(r'\w+')
    unpunctuated_text = tokenizer.tokenize(tweets['text_x_clean'][i])
    tweets['text_x_clean'][i] = " ".join(unpunctuated_text)
    
    #Special Character Replacement -- remove
    for j in range(len(special_characters)):
        tweets['text_x_clean'][i] = tweets['text_x_clean'][i].replace(special_characters[j],' ')
    
    #Lemmatization -- remove
    wordnet_lemmatizer = WordNetLemmatizer()
    text = tweets['text_x_clean'][i]
    lemma_text = nltk.word_tokenize(text)
    tweets['text_x_clean'][i] = " ".join(lemma_text)
    
   
    #Removing Stop Words --remove
    word_tokens = word_tokenize(tweets['text_x_clean'][i])
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    tweets['text_x_clean'][i] = filtered_sentence
    print(tweets['text_x_clean'][i])
'''
    #################################################################################################################
    
    tweets['text_y_clean'][i] = tweets['text_y'][i].lower()
    
    #Conversion of Emojis to Text
    tweets['text_y_clean'][i] = convert_emojis_to_word(tweets['text_y_clean'][i])
    
    #Removal of URLS
    pattern=r'(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))';
    match = re.findall(pattern, tweets['text_y_clean'][i])
    for m in match:
        url = m[0]
        tweets['text_y_clean'][i] = tweets['text_y_clean'][i].replace(url, '')
    
    #Removal of HTTPS
    tweets['text_x_clean'][i] = remove_URL(tweets['text_x_clean'][i])

    #We don't need the following steps of text data preprocessing for deep learning models 
    '''
    #Remove Punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    unpunctuated_text = tokenizer.tokenize(tweets['text_y_clean'][i])
    tweets['text_y_clean'][i] = " ".join(unpunctuated_text)
    
    #Special Character Replacement
    for j in range(len(special_characters)):
        tweets['text_y_clean'][i] = tweets['text_y_clean'][i].replace(special_characters[j],' ')
    
    #Lemmatization
    wordnet_lemmatizer = WordNetLemmatizer()
    text = tweets['text_y_clean'][i]
    lemma_text = nltk.word_tokenize(text)
    tweets['text_y_clean'][i] = " ".join(lemma_text)
    
    
    #Removing Stop Words
    word_tokens = word_tokenize(tweets['text_y_clean'][i])
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    tweets['text_y_clean'][i] = filtered_sentence
    '''
    
    print(tweets['text_y_clean'][i])

    #################################################################################################################
    

<ipython-input-78-58ac437121b3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets['text_x_clean'] = ""
<ipython-input-78-58ac437121b3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets['text_y_clean'] = ""


@115712 hello! we never like our customers to feel like they are not valued.
@115712 i would love the chance to review the account and provide assistance.
@115712 can you please send us a private message, so that i can gain further details about your account?
@115713 h there! we'd definitely like to work with you on this, how long have you been experiencing this issue? -aa
@115715 please send me a private message so that i can send you the link to access your account. -fr
@115716 the information pertaining to the account assumption is correct.  this does need to be done at a local outlet wit... 
@115717 hello, my apologies for any frustrations or inconvenience. i’d be happy to look into this for you? ^mg
@115718 i apologize for the inconvenience. i will be glad to assist you. can you dm me your name and acct # or phone #? -jb
@115719 help has arrived! we are sorry to see that you are having trouble. how can we help?
^hsb
@115720 have your friend message us.
^acm
@115721 are you referri

In [79]:
#Next Steps:
#1) Replace customer id with _sn_
#2) split into train and test
#3)Save to github